In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import git

In [19]:
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [20]:
# import daily dataset
dfd = pd.read_csv(f"{homedir}/data/us/covid/nyt_us_counties_daily.csv", converters={'fips' : str})

In [21]:
# get all counties' fips that require predictions
counties = list(set(list(dfd['fips'])))

In [22]:
# compile initial training dataframe; only including county/date entries after county's date of first death
from tqdm import tqdm

dfX = pd.DataFrame(columns=dfd.columns)
num_counties = len(counties)

for county in tqdm(counties):
    entries = dfd[dfd['fips'] == county]
    start = False
    for index, row in entries.iterrows():
        if row['deaths'] > 0:
            start = True
        if start:
            dfX = dfX.append(row)

100%|██████████████████████████████████████████████████████████████████████████████| 2834/2834 [01:57<00:00, 24.06it/s]


In [24]:
dfX.reset_index()

,index,fips,date,county,state,cases,deaths
0,15592,46087,2020-04-09,McCook,South Dakota,0.0,1.0
1,15593,46087,2020-04-10,McCook,South Dakota,0.0,0.0
2,15594,46087,2020-04-11,McCook,South Dakota,0.0,0.0
3,15595,46087,2020-04-12,McCook,South Dakota,0.0,0.0
4,15596,46087,2020-04-13,McCook,South Dakota,0.0,0.0
5,15597,46087,2020-04-14,McCook,South Dakota,0.0,0.0
6,15598,46087,2020-04-15,McCook,South Dakota,1.0,0.0
7,15599,46087,2020-04-16,McCook,South Dakota,0.0,0.0
8,15600,46087,2020-04-17,McCook,South Dakota,0.0,0.0
9,15601,46087,2020-04-18,McCook,South Dakota,0.0,0.0


In [ ]:
# compile and add entry specific data constructable from current dataframe
import datetime as dt
def date_from_str(ds):
    year = int(ds[:4])
    month = int(ds[5:7])
    day = int(ds[8:])
    return dt.datetime(year, month, day)
# days since date of first death
ddfd = []
# day of week
dw = []
# day of month
dm = []
# deaths one, two, three, four, and five day(s) prior (non-cumulative)
d1 = []
d2 = []
d3 = []
d4 = []
d5 = []
# cases three, four, five, and six weeks prior (non-cumulative)
c3 = []
c4 = []
c5 = []
c5 = []

dates = np.array(dfX['date'])
Xcounties = np.array(dfX['fips'])
deaths = np.array(dfX['deaths'])
curr_county = ""
prev_county = ""
curr_ddfd = 0
curr_date = ""
curr_d1 = 0
curr_d2 = 0
curr_d3 = 0
curr_d4 = 0
curr_d5 = 0
for i in tqdm(range(len(dfX))):
    curr_county = Xcounties[i]
    curr_date = date_from_str(dates[i])
    curr_deaths = 
    if prev_county != curr_county:
        curr_ddfd = 0
        curr_d1 = 0
        curr_d2 = 0
        curr_d3 = 0
        curr_d4 = 0
        curr_d5 = 0
    else:
        curr_d1 = curr_d2
        curr_d2 = curr_d3
        curr_d3 = curr_d4
        curr_d4 = curr_d5
        
        curr_ddfd += 1
        
    ddfd.append(curr_ddfd)
    
    dw.append(curr_date.weekday())
    dm.append(curr_date.day())
    
    
    